In [1]:
import os
import numpy as np
import glob
from tess_sip import SIP
from astropy.coordinates import SkyCoord
from astropy import units as u
import lightkurve as lk
from astroquery.mast import Tesscut
import tess_cpm
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

sys.path.insert(0, "../")
sys.path.insert(0, "../../Packages/eleanor/")
import eleanor

import warnings
warnings.filterwarnings('ignore')

from michael import janet

In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

index = uni[uni.source_id == int(2174289161681454720)].index[0]

In [3]:
j = janet.boot(uni, index=index)


In [4]:
coords = SkyCoord(ra = j.ra, dec = j.dec, unit = (u.deg, u.deg))


In [5]:
from michael.data import data_class
d = data_class(j)
d.setup_data()

Target DR3 ID 2174289161681454720 has tesscut data available on MAST for Sectors 16, 17, 56, 57
Data already downloaded for Sector 16.
Data already downloaded for Sector 17.
Data already downloaded for Sector 56.
Data already downloaded for Sector 57.


In [6]:
d.build_eleanor_lc()

IndexError: index 0 is out of bounds for axis 0 with size 0

In [8]:
d.build_tess_sip_lc(detrended=True)
j.void['rlc_43-44'].plot();
j.void['rdtlc_43-44'].plot();
plt.show()

Running pixels in aperture: 100%|█████████████| 300/300 [00:03<00:00, 91.85it/s]


KeyError: 'rlc_43-44'

In [ ]:
d.build_unpopular_lc()


In [ ]:
ax = j.void['clc_43-44'].plot()
j.void['rdtlc_43-44'].plot(ax=ax)
j.void['cpmlc_43-44'].plot(ax=ax)
plt.show()

In [ ]:
sfiles = []
for sector in j.sectors:
    split = sector.split('-')
    if len(split) > 1:
        sfiles = []
        for s in np.arange(int(split[0]), int(split[1])+1):
            strlen = np.floor(np.log10(s)).astype(int)+1
            secstr = 's0000'[:-strlen] + str(s)

            sfile = glob.glob(f'{d.path}'+
                                f'*{secstr}*astrocut.fits')

            if len(sfile) == 0:
                raise ValueError("No tesscut files could be found for this target.")

            sfiles.append(sfile[0])

        tpflist = [lk.TessTargetPixelFile(f).cutout([26,26],13) for f in sfiles]
        tpfs = lk.TargetPixelFileCollection(tpflist)

In [ ]:
img = tpfs[1].plot();
plt.show()

In [ ]:
lims = img.get_images()[0].get_clim()

In [ ]:
plt.imshow(j.void['datum_44'].tpf[0], origin='lower',
          vmin = lims[0], vmax = lims[1])
pix = np.where(j.void[f'datum_44'].aperture > 0)
plt.scatter(pix[0], pix[1], edgecolors='w', lw=5, marker=',', facecolors='none', s=600, zorder=2, label='Aperture')
plt.show()

In [ ]:
for sfile, s in zip(j.sfiles, j.sectorlist):
    cpm = tess_cpm.Source(sfile, remove_bad=True)
    aperture = j.void[f'datum_{s}'].aperture
    rowlims = 19 + np.array([np.where(aperture>.75)[0].min(), np.where(aperture>.75)[0].max()])
    collims = 19 + np.array([np.where(aperture>.75)[1].min(), np.where(aperture>.75)[1].max()])
    cpm.set_aperture(rowlims = rowlims, collims = collims)

    # We use 200 predictors for a stamp of this size. This is a rough
    # guesstimate from trial-and-error, but seems to work well.
    cpm.add_cpm_model(exclusion_size=6, n=200,
        predictor_method = "similar_brightness")
    cpm.set_regs([0.1])
    cpm.holdout_fit_predict(k=100, verbose=False)

    # Save corrected flux as a lightcurve object for this sector
    flux = cpm.get_aperture_lc(data_type="cpm_subtracted_flux",
                                weighting='median')

In [ ]:
cpm.plot_cutout(show_aperture=True)

In [ ]:
cpm.plot_pix_by_pix()

In [ ]:
plt.plot(flux)
plt.show()